In [ ]:
from autoemulate.simulations.reaction_diffusion import ReactionDiffusion


rd = ReactionDiffusion(n=32, T=10, dt=0.1, return_timeseries=True)
data = rd.forward_samples_spatiotemporal(20)
y = data["data"]

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(y[0, 0, :, :, 0])


In [ ]:
from torch.utils.data import DataLoader
from autoemulate.experimental.data.spatiotemporal_dataset import AutoEmulateDataset

dataset = AutoEmulateDataset(data_path=None, data=data, n_steps_input=1, n_steps_output=1)
batch = next(iter(DataLoader(dataset)))

In [ ]:
batch["input_fields"].shape, batch["output_fields"].shape, batch["constant_scalars"].shape

In [ ]:
batch["input_fields"].shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(batch["input_fields"][0, 0, :, :, 0].cpu())
plt.show()

In [ ]:
from autoemulate.experimental.emulators.fno import FNOEmulator

emulator = FNOEmulator(
    n_modes=(1, 16, 16),
    hidden_channels=16,
    in_channels=3,
    out_channels=1,
)


In [ ]:
# Fit the emulator
emulator.fit(DataLoader(dataset), None)

In [ ]:
# Predictions
y_pred = emulator.predict(DataLoader(dataset), with_grad=False)


In [ ]:
# Evaluate
# TODO: add to emulator perhaps as .evaluate()?
import torch
from autoemulate.experimental.emulators.fno import prepare_batch

y_true = torch.cat(
    [
        prepare_batch(
            batch, channels=(0,), with_constants=True, with_time=True
        )[1]
        for batch in DataLoader(dataset)
    ],
    dim=0
)


In [ ]:
from torchmetrics import R2Score

R2Score()(y_pred.reshape(-1).detach(), y_true.reshape(-1).detach()).item()